In [4]:
from classify import data_pipeline, eco_selector
from resources.get_data import get_current_data
from resources.setup import get_setup
import pandas as pd
import numpy as np
import sys

# Big corpuses

## Rzepa

In [ ]:
corp = 'rzepa'
df = data_pipeline(corp)
df_eco = eco_selector(df)
df_eco.to_csv(f"eco_{corp}.csv")

## Wyborcza

In [ ]:
corp = 'wyborcza'
df = data_pipeline(corp)
df_eco = eco_selector(df, True)
df_eco.to_csv(f"eco_{corp}.csv")

## Gazeta Polska Codziennie

In [ ]:
corp = 'gpc'
df = data_pipeline(corp)
df_eco = eco_selector(df, ease=True)
df_eco.to_csv(f"eco_{corp}.csv")

## Polityka

In [ ]:
corp = 'polityka'
df = data_pipeline(corp)
df_eco = eco_selector(df, ease=True)
df_eco.to_csv(f"eco_{corp}.csv")

# Small Corpuses

## Dorzeczy

In [37]:
corp = 'dorzeczy'
df = data_pipeline(corp)
df_eco = eco_selector(df, True)

{1: 'rzepa', 2: 'gpc', 3: 'newsweek', 4: 'wprost', 5: 'dorzeczy', 6: 'polityka', 7: 'wyborcza', 8: 'wpolityce'}
dorzeczy
Configuration finished
Dataset of 17711 samples
0    8954
1    8757
Name: label, dtype: int64
Preprocessing ended
Accuracy of model 0.9971783295711061


100%|██████████| 2/2 [00:05<00:00,  2.71s/it]


978 368


368

In [38]:
df_eco.to_csv(f"eco_{corp}.csv")

## Wprost

In [24]:
corp = 'wprost'
configuration = get_setup(corp)
df_eco, df_non_eco, df_rest = get_current_data(configuration)
pd.concat([df_eco, df_rest.drop(index=2260)]).to_csv(f"eco_{corp}.csv")

{1: 'rzepa', 2: 'gpc', 3: 'newsweek', 4: 'wprost', 5: 'dorzeczy', 6: 'polityka', 7: 'wyborcza', 8: 'wpolityce'}
wprost


/home/hombre/Code/Climate Project/Classification/resources/get_data.py:7: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_eco = pd.read_csv(drive_path / corpus / "eco_result.csv", index_col=['id', 'source'], parse_dates=['date'])
/home/hombre/Code/Climate Project/Classification/resources/get_data.py:8: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_non_eco = pd.read_csv(drive_path / corpus / "non_eco_result.csv", index_col = ['id', 'source'], parse_dates=['date'])


## Newsweek

In [15]:
corp = 'newsweek'
configuration = get_setup(corp)
df_eco, df_non_eco, df_rest = get_current_data(configuration)
df_all = pd.concat([df_rest['words'].drop(index=2051), df_eco])
df_all.to_csv(f"eco_{corp}.csv")

{1: 'rzepa', 2: 'gpc', 3: 'newsweek', 4: 'wprost', 5: 'dorzeczy', 6: 'polityka', 7: 'wyborcza', 8: 'wpolityce'}
newsweek


## wPolityce

In [23]:
corp = 'wpolityce'
configuration = get_setup(corp)
df_eco, df_non_eco, df_rest = get_current_data(configuration)
df_eco.to_csv(f"eco_{corp}.csv")

{1: 'rzepa', 2: 'gpc', 3: 'newsweek', 4: 'wprost', 5: 'dorzeczy', 6: 'polityka', 7: 'wyborcza', 8: 'wpolityce'}
wpolityce


# Final Processing

In [42]:
import os

def read_and_drop_vectorized(directory_path):
    csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

    dataframes = []
    corpus_len = 0
    tokens = 0
    for csv_file in csv_files:
        file_path = os.path.join(directory_path, csv_file)
        
        df = pd.read_csv(file_path)
        if 'vectorized' in df.columns:
            df = df.drop(columns=['vectorized'])

        if df.index.duplicated().any():
            print(f"Warning: Duplicated index found in file {csv_file}")
            df = df.reset_index()
            df = df.drop(columns=['id'])
            df = df.rename(columns={"index": 'id'})
        
        corpus_len += len(df)
        tokens += df['text'].apply(lambda x: len(x.split(" "))).sum()
        dataframes.append(df)
    print(corpus_len)
    print(tokens)
    return dataframes

# Example usage:
directory_path = '.'
corpus_df = read_and_drop_vectorized(directory_path)


14609
12215235
